# **First Attempt: Utilizing SEC Data via REST API**

In [9]:
"""
reit_type_extractor.py
-----------------------------------------------------------
Functional code to enrich PortfolioGeneralInformation with
INVESTMENTSTYLE = REIT type (Equity / Mortgage / Hybrid).
-----------------------------------------------------------
Author : MSBA 2025 REIT Capstone Team
"""

import re
import time
import requests
import pandas as pd
from typing import List, Dict, Optional
#from snowflake.connector.pandas_tools import write_pandas
#import snowflake.connector as sf

UA = {"User-Agent": "MSBA-2025-Capstone (your.email@univ.edu)"}
TICKER_JSON = "https://www.sec.gov/files/company_tickers.json"
FACTS_URL    = "https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"

# Helper – map Ticker → 10‑digit CIK
def build_ticker_map() -> Dict[str, str]:
    """Download SEC ticker list once per run."""
    master = requests.get(TICKER_JSON, headers=UA, timeout=30).json()
    return {item["ticker"].upper(): str(item["cik_str"]).zfill(10)
            for item in master.values()}

TICKER_TO_CIK = build_ticker_map()

# 2.  Parse 10‑K header text to classify Equity vs Mortgage REIT
MORTGAGE_PAT = re.compile(r"mortgage", re.I)
HYBRID_PAT   = re.compile(r"hybrid", re.I)

def classify_header(header: str) -> str:
    """
    Simple heuristic:
    - if 'Mortgage' appears → Mortgage
    - if 'Hybrid' appears   → Hybrid
    - else                  → Equity (default)
    """
    if MORTGAGE_PAT.search(header):
        return "Mortgage"
    if HYBRID_PAT.search(header):
        return "Hybrid"
    return "Equity"

# Main function
def enrich_reit_type(tickers: List[str],
                     sleep_s: float = 0.25) -> pd.DataFrame:
    """
    Returns a DataFrame with columns:
    TICKER | REITTYPE | INVESTMENTSTYLE
    """
    rows = []
    for tk in tickers:
        tk_u = tk.upper()
        cik = TICKER_TO_CIK.get(tk_u)
        if not cik:
            rows.append({"TICKER": tk_u,
                         "REITTYPE": None,
                         "INVESTMENTSTYLE": None})
            continue

        try:
            facts = requests.get(FACTS_URL.format(cik=cik),
                                 headers=UA, timeout=30).json()
            # Quick sanity: verify SIC = 6798 (public REIT)
            if str(facts.get("sic")) != "6798":
                rows.append({"TICKER": tk_u,
                             "REITTYPE": None,
                             "INVESTMENTSTYLE": None})
                continue

            header_text = facts.get("entityName", "")
            reit_type = classify_header(header_text)
            rows.append({"TICKER": tk_u,
                         "REITTYPE": reit_type,
                         "INVESTMENTSTYLE": reit_type})
        except Exception as exc:
            print(f"[WARN] {tk_u}: {exc}")
            rows.append({"TICKER": tk_u,
                         "REITTYPE": None,
                         "INVESTMENTSTYLE": None})

        time.sleep(sleep_s)            # courtesy pause

    return pd.DataFrame(rows)

# Example usage  (remove or wrap in __main__ for prod)
if __name__ == "__main__":
    sample = ["PLD", "O", "AMT", "NLY", "AAPL"]
    df_reit = enrich_reit_type(sample)
    print(df_reit)
    """
      TICKER  REITTYPE INVESTMENTSTYLE
    0    PLD    Equity          Equity
    1      O    Equity          Equity
    2    AMT    Equity          Equity
    3    NLY  Mortgage        Mortgage
    4   AAPL     None            None
    """

  TICKER REITTYPE INVESTMENTSTYLE
0    PLD     None            None
1      O     None            None
2    AMT     None            None
3    NLY     None            None
4   AAPL     None            None


Nice output, but it returns all as "None". Sanity check: Ticker 'O' is Reality Income aka a publicly traded equity REIT. Therefore, there's an issue in the code. Investigating....

## Sanity Check of API JSON file

In [10]:
import requests
UA = {"User-Agent": "MSBA-2025-Capstone (you@example.edu)"}
url = "https://www.sec.gov/files/company_tickers_exchange.json"
data = requests.get(url, headers=UA).json()
print(type(data))
print(list(data.items())[:1])


<class 'dict'>
[('fields', ['cik', 'name', 'ticker', 'exchange'])]


Looks to be working, let's try the other one.

In [11]:
import requests
UA = {"User-Agent": "MSBA-2025-Capstone (you@example.edu)"}
# Replace {cik} with a valid CIK number for a company
cik_number = "0000320193" # Example CIK for Apple Inc.
url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik_number}.json"
data = requests.get(url, headers=UA).json()
print(type(data))
print(list(data.items())[:1])

<class 'dict'>
[('cik', 320193)]


It works but there is no SIC code! That's why every ticker return's as None - it fails the validation for SIC == 6798, so it falls to SIC != 6798. Troubleshooting...

# **Second Attempt with SEC data via REST API**

In [13]:
"""
reit_type_enrichment_final_structured.py
----------------------------------------------------------------
Pulls SEC data to classify REITs and update PortfolioGeneralInfo.
 - Loads ticker + CIK from exchange file
 - Gets SIC from companyfacts API
 - Classifies REITTYPE = Equity / Mortgage / Hybrid
 - Outputs: PORTFOLIOCODE | INVESTMENTSTYLE
----------------------------------------------------------------
Author: MSBA 2025 Capstone REIT Team
"""

import re, time, requests, pandas as pd
from typing import List

UA = {"User-Agent": "MSBA-2025-Capstone (your.email@university.edu)"}
TICKER_URL = "https://www.sec.gov/files/company_tickers_exchange.json"
FACTS_URL  = "https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"

# Load Ticker + CIK only (no SIC in this file anymore)
def get_ticker_cik_df() -> pd.DataFrame:
    resp = requests.get(TICKER_URL, headers=UA, timeout=30).json()
    fields = resp["fields"]   # ['cik', 'name', 'ticker', 'exchange']
    data   = resp["data"]     # list of [cik, name, ticker, exchange]

    df = pd.DataFrame(data, columns=fields)
    df["TICKER"] = df["ticker"].str.upper()
    df["CIK"] = df["cik"].astype(str).str.zfill(10)
    return df[["TICKER", "CIK"]]

ticker_df = get_ticker_cik_df()

# REIT type classifier from header text
MORTGAGE_PAT = re.compile(r"mortgage", re.I)
HYBRID_PAT   = re.compile(r"hybrid", re.I)

def classify_header(text: str) -> str:
    if MORTGAGE_PAT.search(text):
        return "Mortgage"
    if HYBRID_PAT.search(text):
        return "Hybrid"
    return "Equity"

# Main function
def enrich_reit_type(tickers: List[str], delay=0.25) -> pd.DataFrame:
    rows = []
    for tk in tickers:
        rec = ticker_df[ticker_df["TICKER"] == tk.upper()]
        if rec.empty:
            rows.append({"TICKER": tk.upper(), "REITTYPE": None})
            continue

        cik = rec.iloc[0]["CIK"]
        try:
            facts = requests.get(FACTS_URL.format(cik=cik), headers=UA, timeout=30).json()
            sic = str(facts.get("sic", ""))
            if sic != "6798":  # Not a REIT
                rows.append({"TICKER": tk.upper(), "REITTYPE": None})
                continue

            header = facts.get("entityName", "")
            reit_type = classify_header(header)
        except Exception as e:
            print(f"[WARN] {tk.upper()} – {e}")
            reit_type = None

        rows.append({"TICKER": tk.upper(), "REITTYPE": reit_type})
        time.sleep(delay)
    return pd.DataFrame(rows)

# Example usage and merge to PortfolioGeneralInformation
if __name__ == "__main__":
    tickers = ["PLD", "O", "AMT", "NLY", "AAPL"]
    df_types = enrich_reit_type(tickers)
    print(" REIT Classification:\n", df_types)

    # Simulated ProductMaster table
    product_master = pd.DataFrame({
        "PORTFOLIOCODE": ["ACCT101", "ACCT102", "ACCT103"],
        "TICKER": ["PLD", "O", "NLY"]
    })

    merged = (
        product_master
        .merge(df_types, on="TICKER", how="left")
        .rename(columns={"REITTYPE": "INVESTMENTSTYLE"})
        [["PORTFOLIOCODE", "INVESTMENTSTYLE"]]
    )

    print("\n Final Output Ready for Snowflake Merge:\n", merged)

    # Optional: write_pandas() to Snowflake
    # from snowflake.connector.pandas_tools import write_pandas
    # conn = snowflake.connector.connect(...)
    # write_pandas(conn, merged, "STG_INVESTMENTSTYLE")


 REIT Classification:
   TICKER REITTYPE
0    PLD     None
1      O     None
2    AMT     None
3    NLY     None
4   AAPL     None

 Final Output Ready for Snowflake Merge:
   PORTFOLIOCODE INVESTMENTSTYLE
0       ACCT101            None
1       ACCT102            None
2       ACCT103            None


Still getting all as None :/

Let's try another method

# **Third Attempt**

In [14]:
"""
reittype_lookup.py
Single-source REST API extraction for REITTYPE & ISPUBLICREIT
• Source: SEC EDGAR ‘company_tickers_exchange.json’ for CIK/SIC + ‘companyfacts’ for REIT type
• Handles list of tickers and returns a DataFrame ready for Snowflake
"""
import requests, pandas as pd, re, time
from typing import List, Dict, Optional

UA = {"User-Agent": "MSBA-Capstone-2025 (your@email.edu)"}
# Changed source to company_tickers_exchange.json
TICKER_SIC_SOURCE = "https://www.sec.gov/files/company_tickers_exchange.json"
FACTS_URL         = "https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"

# Build master table with TICKER, CIK, and SIC from company_tickers_exchange.json
def get_ticker_info_map() -> Dict[str, Dict[str, str | None]]:
    """Return a dictionary mapping TICKER to CIK and SIC from company_tickers_exchange.json."""
    resp = requests.get(TICKER_SIC_SOURCE, headers=UA, timeout=30).json()
    # Data is a dictionary with 'fields' and 'data' keys
    fields = resp.get('fields', [])
    raw_data = resp.get('data', [])

    ticker_info_map = {}
    # Find the index of the relevant fields
    try:
        cik_idx = fields.index('cik')
        ticker_idx = fields.index('ticker')
        # SIC is not consistently in this file, handle its absence
        sic_idx = fields.index('sic') if 'sic' in fields else None
    except ValueError as e:
        print(f"[ERROR] Missing expected field in {TICKER_SIC_SOURCE}: {e}")
        return {} # Return empty map if essential fields are missing


    for item in raw_data:
        try:
            cik_str = item[cik_idx]
            ticker = item[ticker_idx]
            sic = item[sic_idx] if sic_idx is not None and sic_idx < len(item) else None # Safely get SIC

            if ticker and cik_str:
                 ticker_info_map[ticker.upper()] = {
                     "CIK": str(cik_str).zfill(10),
                     "SIC": str(sic) if sic is not None else None # Store SIC as string or None
                 }
        except IndexError as e:
            print(f"[WARN] Skipping row due to unexpected structure: {item}. Error: {e}")
            continue # Skip rows that don't match expected structure


    return ticker_info_map

TICKER_INFO_MAP = get_ticker_info_map()

# Simple header classifier
MORT_PAT  = re.compile(r"mortgage", re.I)
HYBRID_PAT= re.compile(r"hybrid", re.I)


def classify_header(text: str) -> str:
    if HYBRID_PAT.search(text):
        return "Hybrid"
    if MORT_PAT.search(text):
        return "Mortgage"
    return "Equity"

# Enrichment function
def enrich_reit_flags(tickers: List[str]) -> pd.DataFrame:
    rows = []
    for t in tickers:
        tk_u = t.upper()
        ticker_info = TICKER_INFO_MAP.get(tk_u)

        if not ticker_info:
            rows.append({"TICKER": tk_u, "REITTYPE": None, "ISPUBLICREIT": False})
            print(f"[INFO] {tk_u}: Ticker info not found in master list from {TICKER_SIC_SOURCE}.")
            continue

        cik = ticker_info.get("CIK")
        sic = ticker_info.get("SIC")

        if sic != "6798":
            rows.append({"TICKER": tk_u, "REITTYPE": None, "ISPUBLICREIT": False})
            # print(f"[INFO] {tk_u}: SIC is {sic}, not 6798. Not a public REIT.") # Optional: uncomment for debugging
            continue

        # If SIC is 6798, it's a public REIT, now classify type from company facts
        try:
            facts = requests.get(FACTS_URL.format(cik=cik),
                                 headers=UA, timeout=15).json()
            registrant = facts.get("entityName", "")
            reit_type = classify_header(registrant)
            rows.append({"TICKER": tk_u, "REITTYPE": reit_type, "ISPUBLICREIT": True})

        except Exception as e:
            print(f"[WARN] {tk_u}: Error fetching company facts for classification: {e}")
            rows.append({"TICKER": tk_u, "REITTYPE": None, "ISPUBLICREIT": True}) # Still a public REIT based on SIC, but type unknown

        time.sleep(0.2)   # SEC rate-limit courtesy

    return pd.DataFrame(rows)

# Quick test
if __name__ == "__main__":
    test = ["PLD", "O", "AMT", "NLY", "AAPL"]   # last one should return False / None
    df = enrich_reit_flags(test)
    print(df)

  TICKER REITTYPE  ISPUBLICREIT
0    PLD     None         False
1      O     None         False
2    AMT     None         False
3    NLY     None         False
4   AAPL     None         False


Clearly, this method isn't working. The main problem is, there isn't a central location where REITs are listed (or any fund for that matter), where the investment style (Equity, Mortgage) is listed. Maybe that's why Assette doesn't have it yet?

Either way using the REST API is helpful for other information but not investment style as it isn't feasibly available for this project's needs (as far as I know right now).

I'm going to try web scraping from another couple of reputable sources (Nareit & REIT Notes) as they have this data more consolidated. Nareit is an excellent resource for all things REIT, so I will try to validate all fund info from there before pulling data from REIT Notes, so Assette can be assured that this data is SEC compliant and reliable.

# **New Approach: Web Scrape from Nareit & REIT Notes**

In [6]:
"""
reit_style_pipeline.py
~~~~~~~~~~~~~~~~~~~~~~
Scrape REITNotes + Nareit, reconcile styles, and return a
DataFrame with columns ['Ticker', 'InvestmentStyle'] where
InvestmentStyle ∈ {'Equity', 'Mortgage', 'Hybrid'}.
"""

import pandas as pd
import requests
from bs4 import BeautifulSoup

UA = {"User-Agent": "Mozilla/5.0 (compatible; MSBA Capstone Bot/1.0)"}


def _fetch_reitnotes() -> pd.DataFrame:
    """Grab the REITNotes master table and keep only ticker & style."""
    url = "https://www.reitnotes.com/reit-list/"
    html = requests.get(url, headers=UA, timeout=30).text
    df = pd.read_html(html)[0][["Symbol", "Type"]].rename(
        columns={"Symbol": "Ticker", "Type": "NotesType"}
    )
    df["NotesType"] = df["NotesType"].str.replace(" REIT", "", regex=False)
    return df


def _fetch_nareit(max_pages: int = 15) -> pd.DataFrame:
    """
    Crawl the REIT Directory.  Any row whose last cell is 'Mortgage'
    is treated as a Mortgage REIT; everything else defaults to Equity.
    Returns columns ['Ticker', 'NareitStyle'].
    """
    rows = []
    for page in range(max_pages):
        url = f"https://www.reit.com/investing/reit-directory?page={page}"
        soup = BeautifulSoup(requests.get(url, headers=UA, timeout=30).text, "html.parser")
        table = soup.find("table")
        if not table:
            break  # no more pages
        for tr in table.tbody.find_all("tr"):
            cells = [td.get_text(strip=True) for td in tr.find_all("td")]
            if len(cells) < 3:
                continue
            ticker = cells[1].split("|")[0].strip()  # 'AHR | Irvine, CA' → 'AHR'
            property_type = cells[-1]
            style = "Mortgage" if property_type.lower() == "mortgage" else "Equity"
            rows.append({"Ticker": ticker, "NareitStyle": style})
    return pd.DataFrame(rows).drop_duplicates()


def classify_styles() -> pd.DataFrame:
    """Main orchestration function."""
    notes = _fetch_reitnotes()
    nareit = _fetch_nareit()

    merged = notes.merge(nareit, on="Ticker", how="outer")

    def _resolve(row):
        a, b = row["NotesType"], row["NareitStyle"]
        if pd.isna(a) and pd.isna(b):
            return None
        if a == b:
            return a  # agreement
        if {"Equity", "Mortgage"} == {a, b}:
            return "Hybrid"  # disagreement → hybrid
        return a or b  # fall back to whichever we have

    merged["InvestmentStyle"] = merged.apply(_resolve, axis=1)
    return merged[["Ticker", "InvestmentStyle"]].dropna().sort_values("Ticker")


# ---------------------------------------------------------------------
if __name__ == "__main__":
    df = classify_styles()
    print(df.head())
    # Example Snowflake load stub (uncomment & configure creds):
    #
    # import snowflake.connector
    # from snowflake.connector.pandas_tools import write_pandas
    #
    # ctx = snowflake.connector.connect(
    #     user="YOUR_USER",
    #     password="YOUR_PASSWORD",
    #     account="YOUR_ACCOUNT",
    #     warehouse="YOUR_WAREHOUSE",
    #     database="YOUR_DB",
    #     schema="YOUR_SCHEMA",
    # )
    # write_pandas(ctx, df, "REIT_INVESTMENT_STYLE", quote_identifiers=False)


/tmp/ipython-input-6-173693241.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0][["Symbol", "Type"]].rename(


    Ticker InvestmentStyle
133    AAT          Equity
134    ABR        Mortgage
135    ACR        Mortgage
136   ACRE        Mortgage
137    ADC          Equity


It worked! Woohoo! This is step in the right direction for the project. Mapping these back to Assette's Snowflake is the next step. Ignore the commented out code...that is some inspo from ChatGPT.

Other considerations to improve: mapping to PortfolioGeneralInformation via primary key = PortfolioCode. 'InvestmentStyle' will be an added column in the table but needs to be mapped to that primary key...will the 'Ticker' column work?

Also - I realized this approach is biased. It is pulling only REIT data so we need to pull the SEC data back in, so Assette can run this code against any ticker in order to populate the PGI table. Let's do that next!

# **Combining Approaches: SEC + Nareit + REIT Notes**

In [15]:
"""
reit_style_pipeline_combined.py
------------------------------------------------------------
Step 1  • SEC master: every US‑listed ticker + CIK
Step 2  • REITNotes scrape: authoritative Equity/Mortgage list
Step 3  • Merge: if ticker in REITNotes keep its style else None
Outputs  • DataFrame [Ticker, InvestmentStyle]
------------------------------------------------------------
"""

import requests
import pandas as pd
from typing import List

UA = {"User-Agent": "Mozilla/5.0 (MSBA Capstone Bot/1.0)"}

# SEC master (tickers + CIK)  – NO SIC here, but we get full universe
def load_sec_master() -> pd.DataFrame:
    url = "https://www.sec.gov/files/company_tickers_exchange.json"
    r   = requests.get(url, headers=UA, timeout=30).json()
    fields = r["fields"]                    # ['cik','name','ticker','exchange']
    df = pd.DataFrame(r["data"], columns=fields)
    df["Ticker"] = df["ticker"].str.upper()
    df["CIK"]    = df["cik"].astype(str).str.zfill(10)
    return df[["Ticker", "CIK"]]

# REITNotes scrape (Ticker, InvestmentStyle)
def load_reitnotes() -> pd.DataFrame:
    url  = "https://www.reitnotes.com/reit-list/"
    html = requests.get(url, headers=UA, timeout=30).text
    notes = (
        pd.read_html(html)[0][["Symbol", "Type"]]
        .rename(columns={"Symbol": "Ticker", "Type": "NotesType"})
    )
    notes["NotesType"] = (
        notes["NotesType"]
        .str.replace(" REIT", "", regex=False)
        .str.strip()
    )
    # Keep only Equity / Mortgage (Hybrid handled later)
    notes["NotesType"] = notes["NotesType"].replace(
        {"Mortgage": "Mortgage", "Equity": "Equity"}
    )
    return notes

# Combine logic
def classify_styles(tickers: List[str]) -> pd.DataFrame:
    sec_df   = load_sec_master()
    notes_df = load_reitnotes()

    # Merge SEC universe with REITNotes on Ticker
    merged = (
        pd.DataFrame({"Ticker": [t.upper() for t in tickers]})
        .merge(sec_df, on="Ticker", how="left")
        .merge(notes_df, on="Ticker", how="left")
    )

    # Where NotesType is missing, it's not a REIT → InvestmentStyle = None
    # REITNotes has only Equity or Mortgage; Hybrid would come from Nareit disagreement logic
    merged["InvestmentStyle"] = merged["NotesType"]

    return merged[["Ticker", "InvestmentStyle"]]

# Demo & Snowflake stub
if __name__ == "__main__":
    sample = ["PLD", "O", "AMT", "NLY", "AAPL"]
    df_style = classify_styles(sample)
    print(df_style)

    # ---> Snowflake load (uncomment & configure)
    # from snowflake.connector import connect
    # from snowflake.connector.pandas_tools import write_pandas
    # conn = connect(...)
    # write_pandas(conn, df_style, "STG_INVESTMENTSTYLE")


  Ticker InvestmentStyle
0    PLD          Equity
1      O          Equity
2    AMT          Equity
3    NLY        Mortgage
4   AAPL             NaN


/tmp/ipython-input-15-565068330.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(html)[0][["Symbol", "Type"]]


## Bringing in Nareit Validation

In [16]:
"""
reit_style_pipeline_hybrid.py
----------------------------------------------------------------
Combines three public sources to classify REIT InvestmentStyle:

  • SEC  : every listed ticker + CIK
  • REITNotes : Equity / Mortgage table
  • Nareit    : Equity / Mortgage inferred from REIT Directory

Logic
------
If ticker in both REITNotes & Nareit:
    • Same style      -> keep that style
    • Different style -> tag HYBRID
If ticker only in one source -> keep that style
If ticker in neither  -> style = None

Outputs DataFrame [Ticker, InvestmentStyle]
----------------------------------------------------------------
pip install pandas requests beautifulsoup4 lxml
"""

import requests, pandas as pd, re, time
from bs4 import BeautifulSoup
from typing import List

UA = {"User-Agent": "Mozilla/5.0 (MSBA Capstone Bot/1.0)"}

# SEC master list (Ticker + CIK only, no style)
def load_sec_master() -> pd.DataFrame:
    url   = "https://www.sec.gov/files/company_tickers_exchange.json"
    data  = requests.get(url, headers=UA, timeout=30).json()
    fields = data["fields"]                 # ['cik','name','ticker','exchange']
    df = pd.DataFrame(data["data"], columns=fields)
    df["Ticker"] = df["ticker"].str.upper()
    df["CIK"]    = df["cik"].astype(str).str.zfill(10)
    return df[["Ticker", "CIK"]]

# REITNotes scrape via pandas.read_html
def load_reitnotes() -> pd.DataFrame:
    url  = "https://www.reitnotes.com/reit-list/"
    html = requests.get(url, headers=UA, timeout=30).text
    notes = (
        pd.read_html(html)[0][["Symbol", "Type"]]
        .rename(columns={"Symbol": "Ticker", "Type": "NotesType"})
    )
    notes["Ticker"] = notes["Ticker"].str.upper()
    notes["NotesType"] = notes["NotesType"].str.replace(" REIT", "", regex=False)
    notes["NotesType"] = notes["NotesType"].map({"Equity": "Equity", "Mortgage": "Mortgage"})
    return notes

# Nareit REIT directory scrape via BeautifulSoup
def load_nareit(max_pages: int = 25) -> pd.DataFrame:
    rows = []
    for page in range(max_pages):
        url = f"https://www.reit.com/investing/reit-directory?page={page}"
        html = requests.get(url, headers=UA, timeout=30).text
        soup = BeautifulSoup(html, "lxml")
        table = soup.find("table")
        if not table or not table.tbody:
            break  # reached the end
        for tr in table.tbody.find_all("tr"):
            tds = [td.get_text(" ", strip=True) for td in tr.find_all("td")]
            if len(tds) < 3:
                continue
            ticker_raw = tds[1].split("|")[0].strip()
            style = "Mortgage" if tds[-1].lower() == "mortgage" else "Equity"
            rows.append({"Ticker": ticker_raw.upper(), "NareitStyle": style})
        time.sleep(0.1)
    return pd.DataFrame(rows).drop_duplicates()

# Resolution logic
def resolve_style(row) -> str | None:
    a, b = row["NotesType"], row["NareitStyle"]
    if pd.isna(a) and pd.isna(b):
        return None
    if a == b:
        return a            # agree (Equity or Mortgage)
    if pd.isna(a):          # only Nareit has value
        return b
    if pd.isna(b):          # only Notes has value
        return a
    return "Hybrid"         # disagree -> Hybrid

# Public function
def classify_styles(tickers: List[str]) -> pd.DataFrame:
    sec     = load_sec_master()
    notes   = load_reitnotes()
    nareit  = load_nareit()

    base = pd.DataFrame({"Ticker": [t.upper() for t in tickers]})
    merged = (
        base.merge(sec,    on="Ticker", how="left")
            .merge(notes,  on="Ticker", how="left")
            .merge(nareit, on="Ticker", how="left")
    )
    merged["InvestmentStyle"] = merged.apply(resolve_style, axis=1)
    return merged[["Ticker", "InvestmentStyle"]]

# Demo + merge to Portfolio code
if __name__ == "__main__":
    sample = ["PLD", "O", "AMT", "NLY", "AAPL"]
    style_df = classify_styles(sample)
    print("Style lookup:\n", style_df)

    # Fake ProductMaster mapping
    product_master = pd.DataFrame({
        "PORTFOLIOCODE": ["ACCT101", "ACCT102", "ACCT103"],
        "Ticker": ["PLD", "O", "NLY"]
    })

    final_df = (
        product_master
        .merge(style_df, on="Ticker", how="left")
        .rename(columns={"InvestmentStyle": "INVESTMENTSTYLE"})
        [["PORTFOLIOCODE", "INVESTMENTSTYLE"]]
    )

    print("\nReady for Snowflake merge:\n", final_df)

    # -- write_pandas Snowflake load example --
    # from snowflake.connector import connect
    # from snowflake.connector.pandas_tools import write_pandas
    # conn = connect(...)
    # write_pandas(conn, final_df, "STG_INVESTMENTSTYLE")


/tmp/ipython-input-16-926238607.py:44: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(html)[0][["Symbol", "Type"]]


Style lookup:
   Ticker InvestmentStyle
0    PLD          Equity
1      O          Equity
2    AMT          Equity
3    NLY        Mortgage
4   AAPL            None

Ready for Snowflake merge:
   PORTFOLIOCODE INVESTMENTSTYLE
0       ACCT101          Equity
1       ACCT102          Equity
2       ACCT103        Mortgage


Let's goooooo :D